# COMP30830 Software Engineering

## Resampling collected weather data

In [ ]:
#from sqlalchemy import create_engine
import sys
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import sqlalchemy as db

In [ ]:
# Will's credentials
# "DB_USER": "admin",
# "DB_PASS": "PFHzvlKMStQpa7BxWLkN",
# "DB_SRVR": "comp30830-wod.cndmh0rmccxq.eu-west-1.rds.amazonaws.com",
# "DB_PORT": "3306",
# "DB_NAME": "dudeWMB",        
    
# Tom's credentials 
# "DB_USER": "tkjswod",
# "DB_PASS": "US5ySn7z5vd1Xqm0AhQw",
# "DB_SRVR": "comp30830-tk.cdkuy8u20d39.eu-west-1.rds.amazonaws.com",
# "DB_PORT": "3306",
# "DB_NAME": "dudeWMB",        
    
# Put temporarily credentials into a dictionary so that it can be used in this notebook.
credentials = {
    "DB_USER": "tkjswod",
    "DB_PASS": "US5ySn7z5vd1Xqm0AhQw",
    "DB_SRVR": "comp30830-tk.cdkuy8u20d39.eu-west-1.rds.amazonaws.com",
    "DB_PORT": "3306",
    "DB_NAME": "dudeWMB"        
}



In [ ]:
connectionString = "mysql+mysqlconnector://" \
            + credentials['DB_USER'] + ":" + credentials['DB_PASS'] \
            + "@" \
            + credentials['DB_SRVR'] + ":" + credentials['DB_PORT']\
            + "/" + credentials['DB_NAME'] + "?charset=utf8mb4"

print(connectionString)

In [ ]:

def getResampleTimeWindowForPreviousHour(dateTimeObj):
    
    if isinstance(dateTimeObj, datetime):        
        print("Trigger time: " + dateTimeObj.strftime("%Y-%m-%d, %H:%M:%S"))
    else:
        print("Error: Datatype 'datatime object' expected ")
        sys.exit()
    
    # If the trigger was just arround midnight, then we need to handle this individually!
    if dateTimeObj.hour == 0:

        # Substract one hour from the provided time
        resampleWindow = dateTimeObj - timedelta(hours = 1)
        # At midnight we need to consider the day when setting WindowStart
        # e.g given time: 00:15:00 --> 23:00:00
        resampleWindowStart = datetime(dateTimeObj.year, dateTimeObj.month, resampleWindow.day, resampleWindow.hour)
        resampleWindowEnd = resampleWindowStart + timedelta(hours = 1)

    # In any other hour but midnight we can merely subract one hour. That's it!
    else:
        # Substract one hour from the provided time
        resampleWindow = dateTimeObj - timedelta(hours = 1)
        # Use the given date but only use previous hour instead of entire time
        # e.g given time: 11:15:00 --> 10:00:00
        resampleWindowStart = datetime(dateTimeObj.year, dateTimeObj.month, dateTimeObj.day, resampleWindow.hour)
        resampleWindowEnd = resampleWindowStart + timedelta(hours = 1)
    
    return [resampleWindowStart, resampleWindowEnd] 


In [ ]:

def resampleWeatherHistoryHourly(connectionString):
    """Function to downsample data 'weather history' to 1 hour samples"""
   
    # Create data-frame variables in outer function so that it can be accessed in inner functions
    df = pd.DataFrame()
    df_resampled = pd.DataFrame()
    
    # That's the time we started collected occopancy data
    firstRecordDateTime = datetime(2022, 2, 21, 12, 35, 27)

    # The create_engine() function produces an Engine object based on a URL
    engine = db.create_engine(connectionString)
   
    #------------------------------------------------------------------------------------------
    # Inner function to delete all rows in resampled table
    #------------------------------------------------------------------------------------------    
    def deleteRowsInResampledTable(dbConnection):
        dbConnection.execute(
            db.text("Delete FROM dudeWMB.dudeWMB.weatherHistoryResampled where weatherHour <> 0;"))

    #------------------------------------------------------------------------------------------
    # Inner function to get date & time of latest sample
    #------------------------------------------------------------------------------------------    
    def getDateTimeLatestSample(dbConnection):

        nonlocal firstRecordDateTime
        
        # Query latest tuple from RDS that is has already been resampled
        result = dbConnection.execute(
        db.text("SELECT * FROM dudeWMB.weatherHistoryResampled ORDER BY weatherHour DESC LIMIT 1;"))
        for tuple in result: # iterate over results - although only one tuple is expected
            dateTimeSample = tuple['weatherHour']
            break
        else:
            # If table 'weatherHistoryResampled' is empty then use date were data collection began
            dateTimeSample = firstRecordDateTime

        return dateTimeSample

    #------------------------------------------------------------------------------------------
    # Inner function to query entire data set
    #------------------------------------------------------------------------------------------    
    def queryEntireData(dbConnection):

        # define variables which aren't local in this inner function
        nonlocal df
        nonlocal engine
        
        # Intialise data frame
        df = pd.DataFrame()   
        # Read entire table 'weatherHistory' and store it in data frame
        df = pd.read_sql_table('weatherHistory', dbConnection)
    
    #------------------------------------------------------------------------------------------
    # Inner function to query data per hour
    #------------------------------------------------------------------------------------------    
    def queryHourlyData(dbConnection, timeObj):
        """Inner function to query data from the 'weatherHistory' table
        
        DRY principle - Don't Repeat Yourself - that's why we user inner functions
        """
        
        # define variables which aren't local in this inner function
        nonlocal df
        nonlocal engine
        
        df = pd.DataFrame()
        
        # Get resampling time window for previous hour as tuple [WindowStart, WindowEnd]
        resampleWindow = getResampleTimeWindowForPreviousHour(timeObj)
        
        # Convert time windows from datetime object to string
        resampleWindowStartStr = resampleWindow[0].strftime("%Y-%m-%d %H:%M:%S")
        resampleWindowStartStr
        resampleWindowEndStr = resampleWindow[1].strftime("%Y-%m-%d %H:%M:%S")
        resampleWindowEndStr
        print("Resample window start: " + str(resampleWindow[0]))
        print("Resample window end: " + str(resampleWindow[1]))
                  
        # Create SQL query string to query data from the 'weatherHistory' table
        sqlQuery = "select * from dudeWMB.weatherHistory where weatherTime > '"\
            + resampleWindowStartStr + "' and weatherTime < '" + resampleWindowEndStr + "'"

        # TODO - implement some error handling using try & execpt
        
        # Query all given data for the specified hour and store in a pandas dataframe 
        df = pd.read_sql(sqlQuery, dbConnection)
        
    #------------------------------------------------------------------------------------------
    # Inner function to resample data per hour 
    #------------------------------------------------------------------------------------------    
    def resampleHourlyData(dbConnection):
        """Inner function to resample the obtained data from the 'weatherHistorystationState' table
        
        DRY principle - Don't Repeat Yourself - that's why we user inner functions
        """
                                      
        # define variables which aren't local in this inner function
        nonlocal df
        nonlocal df_resampled

        # Downsample occupancy data from 30 samples per hour to 1 sample per hour for each station individually

        # Initialise data frame
        df_resampled = pd.DataFrame()

        # Make sure data frame isn't empty
        # This could happen if there is a gap in the data. Then the query would return a empty data frame
        if df.shape[0] != 0:

            df['weatherTime'] = pd.to_datetime(df['weatherTime'])

            df_resampled = df.resample('60min', on='weatherTime').apply(\
                {"latitude":"mean", "longitude":"mean", "description":"first",\
                 "temp":"mean", "humidity":"mean", "wind_speed":"mean"})
            df_resampled = df_resampled.round(\
                {'latitude': 2, 'longitude': 2, 'temp': 2, 'humidity': 2, 'wind_speed': 2})


            print(df_resampled.index)
            print(df_resampled.columns)

#             # Convert the new multiIndex back to a regular index by dropping the top
#             # level and converting it into a column
#             #   -> level=0 only removes level zero from the multiindex
#             # Dropping the first element of the mulitIndex (stationId) and turning it into a column
#             df_resampled = df_resampled.reset_index(level=0, drop=False)
            # Then generate a new column from the index to create new column labeled 'weatherHour'
            df_resampled['weatherHour']=df_resampled.index
            # Now drop the wierd looking index and set it back to numeric
            # --> Getting rid of the weatherTime within the index and turn into a regular index
            df_resampled = df_resampled.reset_index(level=0, drop=True)


            
    #------------------------------------------------------------------------------------------
    # Inner function to write data per hour to RDS
    #------------------------------------------------------------------------------------------    
    def writeHourlyData(dbConnection):
        """Inner function to write resampled data to the 'weatherHistoryResampled' table
        
        DRY principle - Don't Repeat Yourself - that's why we user inner functions
        """

        # define variables which aren't local in this inner function
        nonlocal df_resampled
        
        if df_resampled.shape[0] != 0:
            try:
                with engine.begin() as connection:                   
                    df_resampled.to_sql('weatherHistoryResampled', con=connection,\
                        schema='dudeWMB', index=False, if_exists='append')
                    print(">>> Writing to RDS was successful!")
            except Exception as e:
                print(">>> Something went wrong while writing to RDS!")
                
    #------------------------------------------------------------------------------------------
    # Resampling control
    #------------------------------------------------------------------------------------------    
    with engine.begin() as dbConnection:

        # Get current time
        timeNow = datetime.now()
        print("Time now: " + str(timeNow))
        # Get last sample time
        lastSampleDate = getDateTimeLatestSample(dbConnection)
        print("Last sample date:" + str(lastSampleDate))    

        while lastSampleDate < datetime.now() - timedelta(hours  = 1):

            print("-------------------------------------------------------------")
            lastSampleDate = lastSampleDate + timedelta(hours  = 1)
            queryHourlyData(dbConnection, lastSampleDate)
            display(df)
            resampleHourlyData(dbConnection)
            print("Length data frame resampled: " + str(df_resampled.shape[0]))
            if df_resampled.shape[0] != 0:
                display(df_resampled)
            writeHourlyData(dbConnection)
            print("-------------------------------------------------------------")


                
#                 print("-------------------------------------------------------------")
#                 lastSampleDate = lastSampleDate + timedelta(hours  = 1)
#                 queryHourlyData(dbConnection, lastSampleDate)
#                 display(df)
#                 resampleHourlyData(dbConnection)
#                 print("Length data frame resampled: " + str(df_resampled.shape[0]))
#                 if df_resampled.shape[0] != 0:
#                     display(df_resampled)
#                 writeHourlyData(dbConnection)
#                 print("-------------------------------------------------------------")



In [ ]:
# Driving test code

resampleWeatherHistoryHourly(connectionString)

In [ ]:

# def resampleStationStateHourly(connectionString):
#     """Function to downsample data 'station-state' from 30 samples to 1 sample per hour"""
   
#     # Create data-frame variables in outer function so that it can be accessed in inner functions
#     df = pd.DataFrame()
#     df_resampled = pd.DataFrame()
    
#     # That's the time we started collected occopancy data
#     defaultDateTime = datetime(2022, 2, 22, 12, 53, 25)

#     # The create_engine() function produces an Engine object based on a URL
#     engine = db.create_engine(connectionString)
   
#     #------------------------------------------------------------------------------------------
#     # Inner function to delete all rows in resampled table
#     #------------------------------------------------------------------------------------------    
#     def deleteRowsInResampledTable(dbConnection):
#         dbConnection.execute(
#             db.text("Delete FROM dudeWMB.stationStateResampled where ID <> 0;"))

#     #------------------------------------------------------------------------------------------
#     # Inner function to get date & time of latest sample
#     #------------------------------------------------------------------------------------------    
#     def getDateTimeLatestSample(dbConnection):

#         nonlocal defaultDateTime
        
#         # Query latest tuple from RDS that is has already been resampled
#         result = dbConnection.execute(
#         db.text("SELECT * FROM dudeWMB.stationStateResampled ORDER BY ID DESC LIMIT 1;"))
#         for tuple in result: # iterate over results - although only one tuple is expected
#             dateTimeSample = tuple['weatherHour']
#             break
#         else:
#             # If table 'stationStateResampled' is empty then use date were data collection began
#             dateTimeSample = defaultDateTime
# #             print("Resampled data table is empty, so let's get the first tuple from table 'stationState'.")
# #             result = dbConnection.execute(
# #             db.text("SELECT * FROM dudeWMB.stationState ORDER BY ID LIMIT 1;"))
# #             for tuple in result: # iterate over results - although only one tuple is expected
# #                 dateTimeSample = tuple['weatherTime']

#         return dateTimeSample

#     #------------------------------------------------------------------------------------------
#     # Inner function to query entire data set
#     #------------------------------------------------------------------------------------------    
#     def queryEntireData(dbConnection):

#         # define variables which aren't local in this inner function
#         nonlocal df
#         nonlocal engine
        
#         # Intialise data frame
#         df = pd.DataFrame()   
#         # Read entire table 'stationState' and store it in data frame
#         df = pd.read_sql_table('stationState', dbConnection)
    
#     #------------------------------------------------------------------------------------------
#     # Inner function to query data per hour
#     #------------------------------------------------------------------------------------------    
#     def queryHourlyData(dbConnection, timeObj):
#         """Inner function to query data from the 'stationState' table
        
#         DRY principle - Don't Repeat Yourself - that's why we user inner functions
#         """
        
#         # define variables which aren't local in this inner function
#         nonlocal df
#         nonlocal engine
        
#         df = pd.DataFrame()
        
#         # Get resampling time window for previous hour as tuple [WindowStart, WindowEnd]
#         resampleWindow = getResampleTimeWindowForPreviousHour(timeObj)
        
#         # Convert time windows from datetime object to string
#         resampleWindowStartStr = resampleWindow[0].strftime("%Y-%m-%d %H:%M:%S")
#         resampleWindowStartStr
#         resampleWindowEndStr = resampleWindow[1].strftime("%Y-%m-%d %H:%M:%S")
#         resampleWindowEndStr
#         print("Resample window start: " + str(resampleWindow[0]))
#         print("Resample window end: " + str(resampleWindow[1]))
                  
#         # Create SQL query string to query data from the 'stationState' table
#         sqlQuery = "select * from dudeWMB.stationState where weatherTime > '"\
#             + resampleWindowStartStr + "' and weatherTime < '" + resampleWindowEndStr + "'"

#         # TODO - implement some error handling using try & execpt
        
#         # Query all given data for the specified hour and store in a pandas dataframe 
#         df = pd.read_sql(sqlQuery, dbConnection)
        
#     #------------------------------------------------------------------------------------------
#     # Inner function to resample data per hour 
#     #------------------------------------------------------------------------------------------    
#     def resampleHourlyData(dbConnection):
#         """Inner function to resample the obtained data from the 'stationState' table
        
#         DRY principle - Don't Repeat Yourself - that's why we user inner functions
#         """
                                      
#         # define variables which aren't local in this inner function
#         nonlocal df
#         nonlocal df_resampled

#         # Downsample occupancy data from 30 samples per hour to 1 sample per hour for each station individually

#         df_resampled = pd.DataFrame()

#         if df.shape[0] != 0:

#             df['weatherTime'] = pd.to_datetime(df['weatherTime'])

#             df_resampled = df.groupby(['stationId']).resample('60min', on='weatherTime').apply({"available_bike_stands":"mean", "available_bikes":"mean", "status":"first"})
#             df_resampled = df_resampled.round({'available_bike_stands': 0, 'available_bikes': 0})
#             # Convert the new multiIndex back to a regular index by dropping the top
#             # level and converting it into a column
#             #   -> level=0 only removes level zero from the multiindex
#             # Dropping the first element of the mulitIndex (stationId) and turning it into a column
#             df_resampled = df_resampled.reset_index(level=0, drop=False)
#             # Then generate a new column from the index to create new column labeled 'weatherHour'
#             df_resampled['weatherHour']=df_resampled.index
#             # Now drop the wierd looking index and set it back to numeric
#             # --> Getting rid of the weatherTime within the index and turn into a regular index
#             df_resampled = df_resampled.reset_index(level=0, drop=True)
            

#     #------------------------------------------------------------------------------------------
#     # Inner function to write data per hour to RDS
#     #------------------------------------------------------------------------------------------    
#     def writeHourlyData(dbConnection):
#         """Inner function to write resampled data to the 'stationStateResampled' table
        
#         DRY principle - Don't Repeat Yourself - that's why we user inner functions
#         """

#         # define variables which aren't local in this inner function
#         nonlocal df_resampled
        
#         if df_resampled.shape[0] != 0:
#             try:
#                 with engine.begin() as connection:                   
#                     df_resampled.to_sql('stationStateResampled', con=connection,\
#                         schema='dudeWMB', index=False, if_exists='append')
#                     print(">>> Writing to RDS was successful!")
#             except Exception as e:
#                 print(">>> Something went wrong while writing to RDS!")
                
#     #------------------------------------------------------------------------------------------
#     # Resampling control
#     #------------------------------------------------------------------------------------------    
#     with engine.begin() as dbConnection:

#         # Get current time
#         timeNow = datetime.now()
#         print("Time now: " + str(timeNow))
#         # Get last sample time
#         lastSampleDate = getDateTimeLatestSample(dbConnection)
#         print("Last sample date:" + str(lastSampleDate))    

#         # If no resampled data was found --> start resampling process from the very beginning
#         # defaultDateTime represents the first date & time when we started collecting occupancy data
#         if lastSampleDate == defaultDateTime:

#             # deleteRowsInResampledTable(dbConnection)

#             # Iterate over every day and hour that is listed in the 'stationState' table
#             # and resample each hour individually
#             while lastSampleDate < datetime.now() - timedelta(hours  = 1):
                
#                 lastSampleDate = lastSampleDate + timedelta(hours  = 1)
#                 queryHourlyData(dbConnection, lastSampleDate)
#                 display(df)
#                 resampleHourlyData(dbConnection)
#                 print("Length data frame resampled: " + str(df_resampled.shape[0]))
#                 if df_resampled.shape[0] != 0:
#                     display(df_resampled)
#                 writeHourlyData(dbConnection)

#         # If resampled data was found --> continue resampling on a hourly base
#         else:
#             queryHourlyData(dbConnection, timeNow)
#             display(df)
#             resampleHourlyData(dbConnection)
#             display(df_resampled)
#             writeHourlyData(dbConnection)


